In [1]:
import numpy as np
import pandas as pd
import duckdb
from duckreg import compressed_ols
import pyfixest as pf
from linearmodels.panel import PanelOLS
import statsmodels.api as sm
np.set_printoptions(suppress = True)

## Pooled Models
### Comparison

In [49]:
modis = pd.read_parquet("/scicore/home/meiera/schulz0022/projects/growth-and-temperature/data_nobackup/assembled/modis_subset.parquet")
fit = pf.feols("modis_median ~ ntl_harm", modis)
fit.tidy()

,Estimate,Std. Error,t value,Pr(>|t|),2.5%,97.5%
Coefficient,,,,,,
Intercept,297.407795,0.000451,659196.101376,0.0,297.406910,297.408679
ntl_harm,-0.004571,0.000062,-73.891523,0.0,-0.004692,-0.004450


In [52]:
modis_index = modis.set_index(["pixel_id", "year"])
mod = PanelOLS(modis_index["modis_median"], sm.add_constant(modis_index[["ntl_harm"]]))
res = mod.fit(cov_type='unadjusted')
res.params

/scicore/home/meiera/schulz0022/miniforge-pypy3/envs/gnt/lib/python3.11/site-packages/linearmodels/panel/model.py:1260: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


const       297.407795
ntl_harm     -0.004571
Name: parameter, dtype: float64

These two match exactly.

---

### Compressed OLS

In [2]:
m = compressed_ols(
    formula="modis_median ~ ntl_harm", #pixel_id + year
    data="/scicore/home/meiera/schulz0022/projects/growth-and-temperature/data_nobackup/assembled/modis_subset.parquet",
    n_bootstraps=99,
    round_strata=5,
    seed=42,
    fe_method="mundlak",
    duckdb_kwargs={
        "temp_directory": "/scicore/home/meiera/schulz0022/projects/growth-and-temperature/data_nobackup/assembled/scratch/duckdb_swap",
        "memory_limit": "48GB",
        "max_temp_directory_size": "1024GB",
        "enable_progress_bar": "true"
        },
)
results = m.summary()
restab = pd.DataFrame(
    np.c_[results["point_estimate"], results["standard_error"]],
    columns=["point_estimate", "standard_error"],
)

Starting bootstrap with 99 iterations


100%|██████████| 99/99 [00:00<00:00, 17794.56it/s]


In [3]:
restab

,point_estimate,standard_error
0,297.407795,0.080552
1,-0.004571,0.004171


This one is very different

In [4]:
def wls(X, y, weights=None):
    """Weighted (or unweighted) OLS via numpy.
    X : (n_obs, k)
    y : (n_obs,) or (n_obs, m)
    weights : (n_obs,) non-negative (interpreted as frequency/weight)
    Returns coef array shape (k, m) (m==1 yields (k,1)).
    """
    import numpy as np

    X = np.asarray(X, dtype=float)
    y = np.asarray(y, dtype=float)

    if y.ndim == 1:
        y = y.reshape(-1, 1)

    if weights is None:
        # plain OLS solved via least-squares for numerical stability
        coef, *_ = np.linalg.lstsq(X, y, rcond=None)
        return coef
    # weighted case: pre-multiply by sqrt(weights)
    w = np.asarray(weights).flatten()
    if w.shape[0] != X.shape[0]:
        raise ValueError("weights length must match number of rows in X")
    if np.any(w < 0):
        raise ValueError("weights must be non-negative")
    sqrt_w = np.sqrt(w)
    Xw = X * sqrt_w[:, None]
    yw = y * sqrt_w[:, None]
    coef, *_ = np.linalg.lstsq(Xw, yw, rcond=None)
    return coef

In [61]:
m.df_compressed.sort_values("ntl_harm")

,ntl_harm,count,sum_modis_median,sum_modis_median_sq,mean_modis_median
60,0.0,35988545,1.037612e+10,3.086656e+12,288.317420
48,1.0,38557,1.114808e+07,3.331872e+09,289.132572
41,2.0,7796,2.299376e+06,6.875637e+08,294.943061
40,3.0,596890,1.735231e+08,5.171499e+10,290.712021
19,4.0,2203567,6.421943e+08,1.908067e+11,291.433987
...,...,...,...,...,...
3,59.0,32282,8.669806e+06,2.578355e+09,268.564714
11,60.0,19327,5.184446e+06,1.542305e+09,268.248861
39,61.0,26516,7.069655e+06,2.103265e+09,266.618444
25,62.0,32539,8.578621e+06,2.553707e+09,263.641197


In [64]:
wls(
    np.stack([np.ones(m.df_compressed.shape[0]), m.df_compressed["ntl_harm"].values]).T,
    m.df_compressed["mean_modis_median"].values,
    m.df_compressed["count"].values
)

array([[ 2.88802447e+02],
       [-1.90455450e-01]])

This is the same as the above, so OLS fitting is correct. It must be the compression

In [77]:
df_compressed_manual = modis.dropna(subset = ["ntl_harm", "modis_median"]).groupby("ntl_harm", as_index=False).agg({"pixel_id": "size", "modis_median": "mean"})
df_compressed_manual

,ntl_harm,pixel_id,modis_median
0,0.0,34883481,297.450947
1,1.0,37302,298.860237
2,2.0,7690,299.008596
3,3.0,582283,298.004747
4,4.0,2161681,297.080983
...,...,...,...
59,59.0,29155,297.369443
60,60.0,17429,297.460883
61,61.0,23765,297.481787
62,62.0,28820,297.662071


In [ ]:
wls(
    np.stack([np.ones(df_compressed_manual.shape[0]), df_compressed_manual["ntl_harm"].values]).T,
    df_compressed_manual["modis_median"].values,
    df_compressed_manual["pixel_id"].values
).round(5)

array([[297.40779],
       [ -0.00457]])

In [93]:
con = duckdb.connect()
df_compressed_duckdb = con.sql("""
    SELECT ntl_harm, AVG(modis_median), COUNT(*)
    FROM read_parquet("/scicore/home/meiera/schulz0022/projects/growth-and-temperature/data_nobackup/assembled/modis_subset.parquet/**/*.parquet")
    WHERE ntl_harm IS NOT NULL AND modis_median IS NOT NULL
    GROUP BY ntl_harm
    """).fetchdf()
df_compressed_duckdb

,ntl_harm,avg(modis_median),count_star()
0,47.0,297.407226,19022
1,15.0,297.385141,130960
2,16.0,297.405567,112122
3,27.0,297.444066,36876
4,28.0,297.411831,24288
...,...,...,...
59,14.0,297.248397,115256
60,40.0,297.485709,19460
61,41.0,297.450630,19197
62,45.0,297.386685,16613


In [94]:
wls(
    np.stack([np.ones(df_compressed_duckdb.shape[0]), df_compressed_duckdb["ntl_harm"].values]).T,
    df_compressed_duckdb["avg(modis_median)"].values,
    df_compressed_duckdb["count_star()"].values
).round(5)

array([[297.40779],
       [ -0.00457]])

It is very important when and for what columns NULLS are filtered out. Drop only for strata

---

## FE estimation
### Comparison

In [2]:
modis = pd.read_parquet("/scicore/home/meiera/schulz0022/projects/growth-and-temperature/data_nobackup/assembled/modis_subset.parquet")
modis_subset = modis[["pixel_id", "year", "modis_median", "ntl_harm", "country"]].dropna()
modis_subset["avg_ntl_harm_fe0"] = modis_subset.groupby("pixel_id")["ntl_harm"].transform("mean")
modis_subset["avg_ntl_harm_fe1"] = modis_subset.groupby("year")["ntl_harm"].transform("mean")
modis_subset = modis_subset.round(5)
fit = pf.feols("modis_median ~ ntl_harm | pixel_id + year", modis_subset)
fit.tidy()

/scicore/home/meiera/schulz0022/miniforge-pypy3/envs/gnt/lib/python3.11/site-packages/pyfixest/estimation/model_matrix_fixest_.py:215: UserWarning: 122 singleton fixed effect(s) detected. These observations are dropped from the model.
  warnings.warn(


,Estimate,Std. Error,t value,Pr(>|t|),2.5%,97.5%
Coefficient,,,,,,
ntl_harm,-0.000509,0.00002,-25.903037,0.0,-0.000548,-0.000471


In [5]:
wls(
    np.stack([
        np.ones(modis_subset.shape[0]), 
        modis_subset["ntl_harm"].values,
        modis_subset["avg_ntl_harm_fe0"].values,
        modis_subset["avg_ntl_harm_fe1"].values
        ]).T,
    modis_subset["modis_median"].values
).round(6)

array([[297.260285],
       [ -0.000528],
       [ -0.004687],
       [  0.06669 ]])

### Compressed OLS

In [6]:
m = compressed_ols(
    formula="modis_median ~ ntl_harm | pixel_id + year | 0 | country",
    data="/scicore/home/meiera/schulz0022/projects/growth-and-temperature/data_nobackup/assembled/modis_subset.parquet",
    n_bootstraps=99,
    round_strata=5,
    seed=42,
    fe_method="mundlak",
    duckdb_kwargs={
        "temp_directory": "/scicore/home/meiera/schulz0022/projects/growth-and-temperature/data_nobackup/assembled/scratch/duckdb_swap",
        "memory_limit": "48GB",
        "max_temp_directory_size": "1024GB",
        "enable_progress_bar": "true"
        },
)
results = m.summary()
restab = pd.DataFrame(
    np.c_[results["point_estimate"], results["standard_error"]],
    columns=["point_estimate", "standard_error"],
)
restab

Computing averages for fixed effect dimension 1/2: pixel_id
Computing averages for fixed effect dimension 2/2: year
Creating design matrix with all fixed effect averages


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Compressing data by computing group-level statistics


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Starting bootstrap with 99 iterations


Bootstrap iterations: 100%|██████████| 99/99 [00:05<00:00, 16.93it/s]


,point_estimate,standard_error
0,297.240778,0.398098
1,-0.000677,0.001397
2,-0.004536,0.030849
3,0.074178,0.051430


In [7]:
m.df_compressed.sort_values("count")

,ntl_harm,avg_ntl_harm_fe0,avg_ntl_harm_fe1,count,sum_modis_median,mean_modis_median
411558,47.0,41.28571,2.03377,1,2.979800e+02,297.980000
411559,47.0,41.28571,2.05027,1,2.988800e+02,298.880000
411568,47.0,41.28571,2.51726,1,2.983300e+02,298.330000
411569,47.0,41.28571,2.52512,1,3.009200e+02,300.920000
411570,47.0,41.28571,2.56837,1,2.956400e+02,295.640000
...,...,...,...,...,...,...
3,0.0,0.00000,1.95282,1192800,3.544586e+08,297.165157
1,0.0,0.00000,1.92033,1192803,3.551520e+08,297.745750
5,0.0,0.00000,2.05027,1194506,3.553146e+08,297.457384
4,0.0,0.00000,2.03377,1194552,3.547934e+08,297.009584


In [12]:
wls(
    np.stack([
        np.ones(m.df_compressed.shape[0]), 
        m.df_compressed["ntl_harm"].values,
        m.df_compressed["avg_ntl_harm_fe0"].values,
        m.df_compressed["avg_ntl_harm_fe1"].values
        ]).T,
    m.df_compressed["mean_modis_median"].values,
    m.df_compressed["count"].values,
).round(6)

array([[297.240778],
       [ -0.000677],
       [ -0.004536],
       [  0.074178]])

In [8]:
df_compressed_manual =  modis_subset.groupby(["ntl_harm", "avg_ntl_harm_fe0", "avg_ntl_harm_fe1"], as_index=False).agg({"pixel_id": "size", "modis_median": "mean"})
df_compressed_manual.sort_values("pixel_id")

,ntl_harm,avg_ntl_harm_fe0,avg_ntl_harm_fe1,pixel_id,modis_median
42,0.0,0.05,1.87597,1,295.350000
43,0.0,0.05,1.87602,1,294.710000
44,0.0,0.05,1.89731,1,293.780000
45,0.0,0.05,1.91059,1,294.090000
46,0.0,0.05,2.00775,1,295.000000
...,...,...,...,...,...
3,0.0,0.00,1.91059,1192815,297.165115
0,0.0,0.00,1.87597,1192816,297.745709
6,0.0,0.00,2.02775,1195452,297.456862
5,0.0,0.00,2.00896,1195515,297.008941


In [9]:
m.df_compressed.avg_ntl_harm_fe1.sort_values().unique()

array([1.91031, 1.92033, 1.92967, 1.95282, 2.03377, 2.05027, 2.05077,
       2.09784, 2.23827, 2.27062, 2.30967, 2.39329, 2.41837, 2.47988,
       2.51726, 2.52512, 2.56837, 2.56857, 2.5885 , 2.73124, 3.05691])

In [10]:
df_compressed_manual.avg_ntl_harm_fe1.sort_values().unique()

array([1.87597, 1.87602, 1.89731, 1.91059, 2.00775, 2.00896, 2.02775,
       2.06346, 2.20845, 2.23817, 2.27888, 2.3652 , 2.39156, 2.44904,
       2.47971, 2.49019, 2.53136, 2.53473, 2.55991, 2.69379, 3.03951])

----

In [1]:
import duckdb

con = duckdb.connect("/scicore/home/meiera/schulz0022/projects/growth-and-temperature/data_nobackup/assembled/scratch/demeaned_data.db")

In [2]:
for key, val in {
    "temp_directory": "/scicore/home/meiera/schulz0022/projects/growth-and-temperature/data_nobackup/assembled/scratch/duckdb_swap",
    "memory_limit": "48GB",
    "max_temp_directory_size": "1024GB"
    }.items():
    con.execute(f"SET {key} = '{val}'")

In [9]:
con.sql("""
        CREATE OR REPLACE TABLE design_matrix AS
            SELECT pixel_id, year, modis_median, ix, iy
            FROM read_parquet('/scicore/home/meiera/schulz0022/projects/growth-and-temperature/data_nobackup/assembled/_modis.parquet/**/*.parquet')
        """)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
fe_cols = ["pixel_id", "year"]

In [ ]:
for fe_col in fe_cols:
    con.sql(f"""
    CREATE OR REPLACE TEMP TABLE m_{fe_col} AS
    SELECT {fe_col}, avg(modis_median) AS {fe_col}_avg_modis_median
        FROM read_parquet('/scicore/home/meiera/schulz0022/projects/growth-and-temperature/data_nobackup/assembled/_modis.parquet/**/*.parquet')
        GROUP BY {fe_col};
    """)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
partition_cols = {
    "pixel_id": ["ix"], #, "iy"
    "year": []
}

partition_col_uniques = {col: {} for col in partition_cols.keys()}

for fe_col in partition_col_uniques.keys():
    for batch_col in partition_cols[fe_col]:
        partition_col_uniques[fe_col][batch_col] = con.sql(
            f"""
            SELECT DISTINCT {batch_col}
            FROM read_parquet('/scicore/home/meiera/schulz0022/projects/growth-and-temperature/data_nobackup/assembled/_modis.parquet/**/*.parquet')
            ORDER BY {batch_col}
            """
        ).fetchnumpy()[batch_col].tolist()
        
from itertools import product
iterator = list(product(*partition_col_uniques["pixel_id"].values()))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [43]:
partition_cols = ["ix"]

partition_col_uniques = {col: [] for col in partition_cols}

for partition_col in partition_col_uniques:
    partition_col_uniques[partition_col] = con.sql(
        f"""
        SELECT DISTINCT {partition_col}
        FROM read_parquet('/scicore/home/meiera/schulz0022/projects/growth-and-temperature/data_nobackup/assembled/_modis.parquet/**/*.parquet')
        ORDER BY {partition_col}
        """
    ).fetchnumpy()[partition_col].tolist()
    
from itertools import product
iterator = list(product(*partition_col_uniques.values()))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [45]:
con.sql(f"""
        CREATE OR REPLACE TABLE design_matrix AS
            SELECT pixel_id, year, modis_median, 
            FROM read_parquet('/scicore/home/meiera/schulz0022/projects/growth-and-temperature/data_nobackup/assembled/_modis.parquet/**/*.parquet')
            WITH NO DATA;
        """)

In [46]:
for fe_col in fe_cols:
    con.sql(f"""
            ALTER TABLE design_matrix
                ADD COLUMN {fe_col}_avg_modis_median DOUBLE;
            """)

In [56]:
con.sql(
    """
    SELECT *
    FROM (SELECT *
    FROM read_parquet('/scicore/home/meiera/schulz0022/projects/growth-and-temperature/data_nobackup/assembled/_modis.parquet/**/*.parquet')
        LEFT JOIN m_pixel_id
        USING (pixel_id))
        LEFT JOIN m_year
        USING (year)
    """
)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌───────┬────────────────────┬────────────────────┬────────────────────┬──────────────┬──────────┬─────────┬────────────┬────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬────────────┬───────┬───────┬───────────────────────────┬───────────────────────┐
│ year  │    modis_median    │     modis_mean     │   modis_rollmax3   │ viirs_annual │ ntl_harm │ reg_fav │ nb_mines_a │ nb_diamond │ subdivision │ country │ HDI_ME  │ HDI_VH  │  WB_HI  │  WB_LM  │ HDI_HI  │  WB_UM  │  pixel_id  │  ix   │  iy   │ pixel_id_avg_modis_median │ year_avg_modis_median │
│ int16 │       double       │       double       │       double       │    double    │  double  │ boolean │   float    │   float    │    float    │  float  │ boolean │ boolean │ boolean │ boolean │ boolean │ boolean │   uint64   │ int64 │ int64 │          double           │        double         │
├───────┼────────────────────┼────────────────────┼────────────────────┼──────────────┼──────────┼──

In [55]:
con.sql(
    """
    SELECT *
    FROM read_parquet('/scicore/home/meiera/schulz0022/projects/growth-and-temperature/data_nobackup/assembled/_modis.parquet/**/*.parquet')
        LEFT JOIN m_pixel_id
        USING (pixel_id)
    """
)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌───────┬────────────────────┬────────────────────┬────────────────────┬──────────────┬──────────┬─────────┬────────────┬────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬────────────┬───────┬───────┬───────────────────────────┐
│ year  │    modis_median    │     modis_mean     │   modis_rollmax3   │ viirs_annual │ ntl_harm │ reg_fav │ nb_mines_a │ nb_diamond │ subdivision │ country │ HDI_ME  │ HDI_VH  │  WB_HI  │  WB_LM  │ HDI_HI  │  WB_UM  │  pixel_id  │  ix   │  iy   │ pixel_id_avg_modis_median │
│ int16 │       double       │       double       │       double       │    double    │  double  │ boolean │   float    │   float    │    float    │  float  │ boolean │ boolean │ boolean │ boolean │ boolean │ boolean │   uint64   │ int64 │ int64 │          double           │
├───────┼────────────────────┼────────────────────┼────────────────────┼──────────────┼──────────┼─────────┼────────────┼────────────┼─────────────┼─────────┼─────────┼────

In [53]:
from tqdm.notebook import tqdm
for it in tqdm(iterator):
    where_clause = " AND ".join([f"{partition_cols[i]} == {it[i]}" for i in range(len(it))])
    mean_variables = ", ".join([f"{fe_col}_avg_modis_median" for fe_col in fe_cols])
    join_statements = "\n".join([f"LEFT JOIN m_{fe_col} USING ({fe_col})" for fe_col in fe_cols])
    
    con.sql(f"""
        INSERT INTO design_matrix BY NAME (
            SELECT pixel_id, year, modis_median, {mean_variables}
                FROM read_parquet('/scicore/home/meiera/schulz0022/projects/growth-and-temperature/data_nobackup/assembled/_modis.parquet/**/*.parquet')
                {join_statements}
                WHERE {where_clause}
        );
        """)
            

  0%|          | 0/17 [00:00<?, ?it/s]

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Error: KeyboardInterrupt: <EMPTY MESSAGE>

At:
  /scicore/home/meiera/schulz0022/miniforge-pypy3/envs/gnt/lib/python3.11/site-packages/traitlets/traitlets.py(708): __set__
  /scratch/ipykernel_1069269/1061293578.py(7): <module>
  /scicore/home/meiera/schulz0022/miniforge-pypy3/envs/gnt/lib/python3.11/site-packages/IPython/core/interactiveshell.py(3672): run_code
  /scicore/home/meiera/schulz0022/miniforge-pypy3/envs/gnt/lib/python3.11/site-packages/IPython/core/interactiveshell.py(3612): run_ast_nodes
  /scicore/home/meiera/schulz0022/miniforge-pypy3/envs/gnt/lib/python3.11/site-packages/IPython/core/interactiveshell.py(3367): run_cell_async
  /scicore/home/meiera/schulz0022/miniforge-pypy3/envs/gnt/lib/python3.11/site-packages/IPython/core/async_helpers.py(128): _pseudo_sync_runner
  /scicore/home/meiera/schulz0022/miniforge-pypy3/envs/gnt/lib/python3.11/site-packages/IPython/core/interactiveshell.py(3155): _run_cell
  /scicore/home/meiera/schulz0022/miniforge-pypy3/envs/gnt/lib/pytho

In [ ]:
from tqdm.notebook import tqdm
for fe_col in partition_cols.keys():
    if partition_cols[fe_col]:
        for it in tqdm(iterator):
            where_clause = " AND ".join([f"{partition_cols[fe_col][i]} == {it[i]}"for i in range(len(it))])
            
            con.sql(f"""
                CREATE OR REPLACE TEMP TABLE m1 AS
                SELECT {fe_col}, avg(modis_median) AS {fe_col}_avg_modis_median
                    FROM read_parquet('/scicore/home/meiera/schulz0022/projects/growth-and-temperature/data_nobackup/assembled/_modis.parquet/**/*.parquet')
                    WHERE {where_clause}
                    GROUP BY {fe_col};
                    
                INSERT INTO design_matrix BY NAME (
                    SELECT pixel_id, year, modis_median, {fe_col}_avg_modis_median, ix, iy
                        FROM read_parquet('/scicore/home/meiera/schulz0022/projects/growth-and-temperature/data_nobackup/assembled/_modis.parquet/**/*.parquet')
                            LEFT JOIN m1
                            USING (pixel_id)
                        WHERE {where_clause}
                );
                """)
            

  0%|          | 0/17 [00:00<?, ?it/s]

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Error: KeyboardInterrupt: <EMPTY MESSAGE>

At:
  /scicore/home/meiera/schulz0022/miniforge-pypy3/envs/gnt/lib/python3.11/site-packages/traitlets/traitlets.py(708): __set__
  /scratch/ipykernel_1069269/466942280.py(7): <module>
  /scicore/home/meiera/schulz0022/miniforge-pypy3/envs/gnt/lib/python3.11/site-packages/IPython/core/interactiveshell.py(3672): run_code
  /scicore/home/meiera/schulz0022/miniforge-pypy3/envs/gnt/lib/python3.11/site-packages/IPython/core/interactiveshell.py(3612): run_ast_nodes
  /scicore/home/meiera/schulz0022/miniforge-pypy3/envs/gnt/lib/python3.11/site-packages/IPython/core/interactiveshell.py(3367): run_cell_async
  /scicore/home/meiera/schulz0022/miniforge-pypy3/envs/gnt/lib/python3.11/site-packages/IPython/core/async_helpers.py(128): _pseudo_sync_runner
  /scicore/home/meiera/schulz0022/miniforge-pypy3/envs/gnt/lib/python3.11/site-packages/IPython/core/interactiveshell.py(3155): _run_cell
  /scicore/home/meiera/schulz0022/miniforge-pypy3/envs/gnt/lib/python

In [ ]:
con.sql("SELECT avg(modis_median) OVER (PARTITION BY pixel_id) FROM read_parquet('/scicore/home/meiera/schulz0022/projects/growth-and-temperature/data_nobackup/assembled/modis.parquet/**/*.parquet') LIMIT 1000").show()

In [4]:
con.sql("""
        SELECT *
        FROM m1
        """).show()

┌─────────────────┬────────────────────┐
│    pixel_id     │  avg_modis_median  │
│     uint64      │       double       │
├─────────────────┼────────────────────┤
│     47244860234 │   259.562380952381 │
│     47244860519 │  258.2338095238095 │
│     47244863566 │ 260.02809523809526 │
│     47244863781 │  260.1257142857143 │
│     47244863900 │ 260.12761904761896 │
│     47244863955 │  259.7704761904762 │
│     47244864355 │ 258.67952380952374 │
│     47244864429 │ 259.12428571428575 │
│     47244865781 │  259.5266666666667 │
│     47244865923 │  260.1580952380952 │
│          ·      │          ·         │
│          ·      │          ·         │
│          ·      │          ·         │
│ 562992905044503 │  273.5495238095238 │
│ 562992905044568 │ 273.44047619047615 │
│ 562992905045043 │  273.1819047619048 │
│ 562992905045167 │  273.0904761904761 │
│ 562992905045236 │ 272.99142857142857 │
│ 562992905045363 │  273.1176190476191 │
│ 562992905045546 │ 273.13238095238097 │
│ 56299290504569

In [ ]:
con.sql("""
        CREATE OR REPLACE TABLE demeaned AS
            SELECT pixel_id, year, modis_median - m1.avg_modis_median
            FROM read_parquet('/scicore/home/meiera/schulz0022/projects/growth-and-temperature/data_nobackup/assembled/modis.parquet/**/*.parquet') d
                LEFT JOIN m1 USING (pixel_id)
        """)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
con.sql("""
        SELECT modis_median - avg(modis_median) OVER (PARTITION BY pixel_id) FROM read_parquet('/scicore/home/meiera/schulz0022/projects/growth-and-temperature/data_nobackup/assembled/modis.parquet/**/*.parquet')
        """).show()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
con.sql("SELECT avg(modis_median) OVER (PARTITION BY pixel_id) FROM read_parquet('/scicore/home/meiera/schulz0022/projects/growth-and-temperature/data_nobackup/assembled/modis.parquet/**/*.parquet') LIMIT 1000").show()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [41]:
duckdb.read_parquet("/scicore/home/meiera/schulz0022/projects/growth-and-temperature/data_nobackup/assembled/_modis.parquet/**/*.parquet")

┌───────┬──────────────┬────────────┬────────────────┬──────────────┬──────────┬─────────┬────────────┬────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬──────────┐
│ year  │ modis_median │ modis_mean │ modis_rollmax3 │ viirs_annual │ ntl_harm │ reg_fav │ nb_mines_a │ nb_diamond │ subdivision │ country │ HDI_ME  │ HDI_VH  │  WB_HI  │  WB_LM  │ HDI_HI  │  WB_UM  │ pixel_id │
│ int16 │    double    │   double   │     double     │    double    │  double  │ boolean │   float    │   float    │    float    │  float  │ boolean │ boolean │ boolean │ boolean │ boolean │ boolean │  uint64  │
├───────┼──────────────┼────────────┼────────────────┼──────────────┼──────────┼─────────┼────────────┼────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┼─────────┼─────────┼─────────┼──────────┤
│  2000 │         NULL │       NULL │           NULL │         NULL │      0.0 │ false   │       NULL │       NULL │        NULL │    NULL │ false   │ f

In [28]:
con.sql("SELECT count() FROM read_parquet('/scicore/home/meiera/schulz0022/projects/growth-and-temperature/data_nobackup/assembled/modis.parquet/**/*.parquet')").show()

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│  17871105294 │
└──────────────┘



In [ ]:
con.sql("SELECT count() FROM read_parquet('/scicore/home/meiera/schulz0022/projects/growth-and-temperature/data_nobackup/assembled/_modis.parquet/**/*.parquet') WHERE ntl_harm IS NOT NULL").show()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│  12201999306 │
└──────────────┘



In [11]:
con.sql("SELECT round(viirs_annual, 5), COUNT() FROM read_parquet('/scicore/home/meiera/schulz0022/projects/growth-and-temperature/data_nobackup/assembled/modis_subset.parquet/**/*.parquet') GROUP BY round(viirs_annual, 5)").show()
# ROUND_EVEN(modis_median, 5)

┌────────────────────────┬──────────────┐
│ round(viirs_annual, 5) │ count_star() │
│         double         │    int64     │
├────────────────────────┼──────────────┤
│                   0.31 │        39122 │
│                   0.52 │        25305 │
│                   0.11 │        24273 │
│                   0.46 │        29761 │
│                   1.24 │         5957 │
│                   1.09 │         7556 │
│                  16.04 │           40 │
│                   8.08 │          168 │
│                   2.04 │         2458 │
│                   1.19 │         6442 │
│                     ·  │            · │
│                     ·  │            · │
│                     ·  │            · │
│                  75.02 │            2 │
│                  22.55 │           35 │
│                  19.87 │           37 │
│                  53.12 │            6 │
│                  44.28 │           11 │
│                  50.74 │           10 │
│                  58.33 │        

In [13]:
con.sql("SELECT avg(modis_median) OVER (PARTITION BY pixel_id) FROM read_parquet('/scicore/home/meiera/schulz0022/projects/growth-and-temperature/data_nobackup/assembled/modis.parquet/**/*.parquet')").show()


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

: 

: 

: 

---

In [7]:
import duckdb

con = duckdb.connect("")
file = "read_parquet('/scicore/home/meiera/schulz0022/projects/growth-and-temperature/data_nobackup/assembled/__modis.parquet/**/*.parquet')"

In [8]:
con.sql(
    f"""
    SELECT count(*)
    FROM {file}
    """
)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│  17825197929 │
└──────────────┘

In [9]:
con.sql(
    """
    SELECT *
    FROM read_parquet('/scicore/home/meiera/schulz0022/projects/growth-and-temperature/data_nobackup/assembled/_modis.parquet/**/*.parquet')
    """
)

┌───────┬──────────────┬────────────┬────────────────┬──────────────┬──────────┬─────────┬────────────┬────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬──────────┬───────┬───────┐
│ year  │ modis_median │ modis_mean │ modis_rollmax3 │ viirs_annual │ ntl_harm │ reg_fav │ nb_mines_a │ nb_diamond │ subdivision │ country │ HDI_ME  │ HDI_VH  │  WB_HI  │  WB_LM  │ HDI_HI  │  WB_UM  │ pixel_id │  ix   │  iy   │
│ int16 │    double    │   double   │     double     │    double    │  double  │ boolean │   float    │   float    │    float    │  float  │ boolean │ boolean │ boolean │ boolean │ boolean │ boolean │  uint64  │ int64 │ int64 │
├───────┼──────────────┼────────────┼────────────────┼──────────────┼──────────┼─────────┼────────────┼────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┼─────────┼─────────┼─────────┼──────────┼───────┼───────┤
│  2000 │         NULL │       NULL │           NULL │         NULL │      0.0 │ false  

17871105294, all columns, read problems
_ 17871105294, all columns
__ 17825197929